In [9]:
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import os
class BankAccount:
    def __init__(self, username):
        self.username = username
        self.balance = self.load_balance()

    def load_balance(self):
        if os.path.exists("accounts.txt"):
            with open("accounts.txt", "r") as file:
                for line in file:
                    user, balance = line.strip().split("|")
                    if user == self.username:
                        return float(balance)
        return 0.0

    def save_balance(self):
        accounts = {}
        if os.path.exists("accounts.txt"):
            with open("accounts.txt", "r") as file:
                for line in file:
                    user, balance = line.strip().split("|")
                    accounts[user] = balance

        accounts[self.username] = str(self.balance)
        with open("accounts.txt", "w") as file:
            for user, balance in accounts.items():
                file.write(f"{user}|{balance}\n")

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            self.save_balance()
            self.record_transaction(f"Deposited: ${amount}")
            print(f"✅ Deposit successful! New Balance: ${self.balance}")
        else:
            print("❌ Invalid deposit amount.")

    def withdraw(self, amount):
        if amount > 0 and amount <= self.balance:
            self.balance -= amount
            self.save_balance()
            self.record_transaction(f"Withdrew: ${amount}")
            print(f"✅ Withdrawal successful! New Balance: ${self.balance}")
        else:
            print("❌ Insufficient balance or invalid amount.")

    def send_money(self, recipient, amount):
        if amount > 0 and amount <= self.balance:
            recipient_account = BankAccount(recipient)
            self.balance -= amount
            recipient_account.balance += amount
            self.save_balance()
            recipient_account.save_balance()

            self.record_transaction(f"Sent ${amount} to {recipient}")
            recipient_account.record_transaction(f"Received ${amount} from {self.username}")
            print(f"✅ Transfer successful! Sent ${amount} to {recipient}")
        else:
            print("❌ Transfer failed. Insufficient balance or invalid amount.")

    def record_transaction(self, message):
        with open("transactions.txt", "a") as file:
            file.write(f"{self.username}: {message}\n")

    def view_transaction_history(self):
        history = ""
        if os.path.exists("transactions.txt"):
            with open("transactions.txt", "r") as file:
                for line in file:
                    if line.startswith(self.username):
                        history += line.strip() + "\n"

        if history:
            messagebox.showinfo("Transaction History", history)
        else:
            messagebox.showinfo("Transaction History", "No transactions found.")

def authenticate():
    username = username_entry.get().strip()
    password = password_entry.get().strip()

    if not username or not password:
        messagebox.showerror("Login Failed", "Username and Password cannot be empty!")
        return

    if os.path.exists("users.txt"):
        with open("users.txt", "r") as file:

            for line in file:
                stored_username, stored_password = line.strip().split("|")
                if username == stored_username and password == stored_password:
                    messagebox.showinfo("Login Successful", f"Welcome, {username}!")
                    root.destroy()  # Close login window
                    open_banking_menu(username)  # Open banking menu
                    return

    messagebox.showerror("Login Failed", "Invalid username or password!")

# Open Banking Menu After Login
def open_banking_menu(username):
    account = BankAccount(username)

    banking_window = tk.Tk()
    banking_window.title(f"Banking System - {username}")

    tk.Label(banking_window, text="Banking Menu", font=("Arial", 14)).pack(pady=10)

    tk.Button(banking_window, text="Deposit", command=lambda: deposit_money(account)).pack(pady=5)
    tk.Button(banking_window, text="Withdraw", command=lambda: withdraw_money(account)).pack(pady=5)
    tk.Button(banking_window, text="Send Money", command=lambda: send_money(account)).pack(pady=5)
    tk.Button(banking_window, text="Check Balance", command=lambda: messagebox.showinfo("Balance", f"Current Balance: ${account.balance}")).pack(pady=5)
    tk.Button(banking_window, text="Transaction History", command=lambda: account.view_transaction_history()).pack(pady=5)
    tk.Button(banking_window, text="Exit", command=banking_window.destroy).pack(pady=10)

    banking_window.mainloop()

# Deposit Money Function
def deposit_money(account):
    amount = simpledialog.askfloat("Deposit", "Enter amount to deposit:")
    if amount is not None:
        account.deposit(amount)

def withdraw_money(account):
    amount = simpledialog.askfloat("Withdraw", "Enter amount to withdraw:")
    if amount is not None:
        account.withdraw(amount)

def send_money(account):
    recipient = simpledialog.askstring("Send Money", "Enter recipient's username:")
    if recipient:
        amount = simpledialog.askfloat("Send Money", f"Enter amount to send to {recipient}:")
        if amount is not None:
            account.send_money(recipient, amount)

# UI for Login
root = tk.Tk()
root.title("Login Page")

tk.Label(root, text="Username:").pack(pady=5)
username_entry = tk.Entry(root)
username_entry.pack(pady=5)

tk.Label(root, text="Password:").pack(pady=5)
password_entry = tk.Entry(root, show="*")
password_entry.pack(pady=5)

tk.Button(root, text="Login", command=authenticate).pack(pady=10)

root.mainloop()



